<a href="https://colab.research.google.com/github/pramit46/LLMTry/blob/main/Local_Persistence/LLM_RAG_Local_Persistence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Dependencies**

In [22]:
!pip -qU install pypdf
## Embedding
!pip install -qU sentence_transformers
!pip install -qU transformers einops accelerate langchain bitsandbytes

!pip install -qU llama_index
!pip install -qU llama-index-llms-huggingface
!pip install -qU llama-index-embeddings-langchain
!pip install -qU langchain-community
!pip install -qU chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.4 MB/s eta 0:00:00
  

**All Imports. Needs cleanup**

In [9]:
import llama_index
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.core.prompts.prompts import SimpleInputPrompt
from llama_index.llms.huggingface import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader

**Load the Inputs File**

In [13]:
#documents=SimpleDirectoryReader("/content/").load_data()

file_path="/content/446012636724.pdf"
loader = PyPDFLoader(file_path)
documents = []
async for page in loader.alazy_load():
    documents.append(page)

**Verification of Input File Content**

In [14]:
documents.__class__.__name__
print(documents[0].page_content)

         SUBHASPALLY CUSTOMER CARE CENTER, PHONE No - 3532596057(24 HRS)
 1 NO. DABGRAM COLONY,  CALL CENTER PHONE No - 19121(TOLL FREE), TAN: CALW05053G
 PRABIR KR. BASU. RABINDRA NAGAR.         Invoice No.          : 446012636724
 ATTACHAD WITH RABINDRA SANGHA. P.0.-     Prev. Reading Date   : 20.03.2023
 R/ SARANI. DT- SILIGURI.                 Present Reading Date : 03.07.2023
 Pin -                                    Billing Date         : 03.07.2023
 Consumer Id        :   412205640         Next Reading Date:18.09.2023-22.09.2023
 Tariff Class       :   A(DM-U)           Connected Load       : 2.24 KVA
 Installation No    :   572997            Solar PV Capacity    :
 Latitude           :   26.7111425        Meter Reading unit   : GET10QMR
 Longitude          :   88.439323         PAN of consumer(s)   :
 ________________________________________________________________________________
 Meter No  Time  Previous   Present          MF      Unit              Max Demand
                

**Create Embedding Function**

In [38]:
from langchain.embeddings import SentenceTransformerEmbeddings
embedding=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


**Create Persist Directory and Persist. The persist directory is created within the colab directory only. May need to download manually**

In [63]:
persist_dir="C:\\Users\\PramitBasu\\OneDrive - kyndryl\\DownloadX\\Personal\\ML_Try\\PersistDir"
docs_with_embedding=Chroma.from_documents(documents=documents,embedding=embedding, persist_directory=persist_dir)
docs_with_embedding.persist()

**Use similarity_search to find out how many documents are relevant to the query. It's irrelevant here since there is only one document**

In [60]:
query="what is consumer Id?"
doc=docs_with_embedding.similarity_search(query)

AttributeError: 'list' object has no attribute 'persist'

**Verify the Class of the instance doc and docs_with_embedding**

In [73]:
print(doc.__class__.__name__)
print(docs_with_embedding.__class__.__name__)

list
Chroma


**As part of the persistence testing, nullify the embed object**

In [66]:
docs_with_embedding=None

**After Nullifying, the get() should not return anything other than exception**

In [67]:
docs_with_embedding.get()

AttributeError: 'NoneType' object has no attribute 'get'

**Pull back the data from the persistent directory and reembed**

In [69]:
docs_with_embedding=Chroma(persist_directory=persist_dir, embedding_function=embedding)

Runt he get() again, and this time it should show the data

In [71]:
docs_with_embedding.get()

{'ids': ['19c6a2f0-c9db-4c0a-b616-78fd1e267905',
  '026f4798-73c3-4cda-8ea8-95038f93d381',
  '459e2ffb-aa2e-4f76-b5a0-88c814c02646'],
 'embeddings': None,
 'metadatas': [{'page': 0, 'source': '/content/446012636724.pdf'},
  {'page': 0, 'source': '/content/446012636724.pdf'},
  {'page': 0, 'source': '/content/446012636724.pdf'}],
 'documents': ['         SUBHASPALLY CUSTOMER CARE CENTER, PHONE No - 3532596057(24 HRS)\n 1 NO. DABGRAM COLONY,  CALL CENTER PHONE No - 19121(TOLL FREE), TAN: CALW05053G\n PRABIR KR. BASU. RABINDRA NAGAR.         Invoice No.          : 446012636724\n ATTACHAD WITH RABINDRA SANGHA. P.0.-     Prev. Reading Date   : 20.03.2023\n R/ SARANI. DT- SILIGURI.                 Present Reading Date : 03.07.2023\n Pin -                                    Billing Date         : 03.07.2023\n Consumer Id        :   412205640         Next Reading Date:18.09.2023-22.09.2023\n Tariff Class       :   A(DM-U)           Connected Load       : 2.24 KVA\n Installation No    :   57299